# 本チャプターの目次
1. ビジネスメタデータの重要な役割
2. ビジネスメタデータの活用方法
3. 代表的なビジネスメタデータは？
4. 実際にビジネスメタデータを覗いてみよう
5. メタデータ保存用のテーブルを作成してみよう

# ビジネスメタデータとは？
ビジネスメタデータの

# 代表的なビジネスメタデータは？
代表的なビジネスメタデータはテーブル定義です。  
テーブル定義といっても、テーブル名やデータベース名、カラム名と言ったオーソドックスなところから、  
各カラムの説明といったビジネス要件などの説明を含みます。  

ビジネス要件（ドメイン知識などともよばれる）は社内の慣例や業界の慣例、システムの特別な仕様など、見ただけではわからないような仕様のことを指しています。  
特にビジネス要件は機械的には抽出することができず人の手に頼っているのが実情です。  

そのため、ビジネスメタデータは単純にテーブル定義だけを示せば良いわけではなく単一システムだけにとどまらない、テーブル同士やシステム同士の紐付きを言語化し登録する必要もあるというわけになります（リネージュと呼んだりします）。  

そのため現場のエンジニアだけで作業が完了する問いことはまずなく、データ提供〜データ分析に参画する全てのユーザがこのビジネスメタデータに参画する必要があります。  

# ビジネスメタデータの重要性や活用方法
ビジネスメタデータは、従来から存在しているテーブル定義に対してより付加価値をつけていくものと言えるわけなのですが、


# 実際にビジネスメタデータを覗いてみよう
ここからは実際にビジネスメタデータ(テーブル定義)を覗いてみましょう。

復習になりますが、メタデータはメタデータストア(今回だとMysqlに保存していきます。)
特にテーブル定義はいづれの環境（オンプレでも、クラウドでも）自動的に保存されることが多いので、最初に確認するにはちょうどいい題材です。

セクション２で行った環境構築設定などありますので、接続情報など忘れてしまった方は、以下のノートブックを見直してみてください。
```
必要ツールの設定を行います。
以下を参照いただきご自身の環境作成をおこないます
https://github.com/yk-st/pyspark_settings/blob/main/setting.ipynb
```

## Mysqlに接続してみよう
1. mysql -uroot -proot でデータベースに接続
2. use metastore　でmetastoreデータベースに接続
3. show tablesでテーブル一覧を表示
4. select * from TBL でメタデータストア内に保存されているテーブルを取得

jinko_table
sample_metadataテーブル共にセクション３で作成したSparkテーブルです。

5. select * from DBSでデータベース情報を確認してみましょう

DB_IDが4で取得した値と一致していることが確認できるはずです

6. 肝心の定義はどこにあるでしょうか？
SD_IDをもとにテーブル定義を引っ張ってきています。
少し説明しにくいのでSQLを見て下さい。
以下のSQLを実行するとsmaple_metadataテーブルのカラム名をメタデータストアから取得するSQLです。

```
select co.COLUMN_NAME from TBLS t 
   inner join SDS as s on t.SD_ID=s.SD_ID 
   inner join CDS c on s.CD_ID=c.CD_ID 
   inner join COLUMNS_V2 co on c.CD_ID = co.CD_ID 
where TBL_NAME='sample_metadata';
```

Mysqlで構築する場合特有の話になってしまうのであまり踏み込みませんが、全てのSPARKテーブルに関する情報は全て上記のようにJOINをすることによって取得することが可能です。  
じつ業務で活用する場合は、上記に加えてカラムの説明を保存するテーブルを作成(今回の場合はsample_metadataテーブル)して結合することで統合的にGUIやAPIで表示返却する  
ということが行われます。  

# メタデータ保存用のテーブルを作成してみよう

最終的にメタデータをMysqlに保存する目的で本コースでは以下の順番で対応していきます。

1. テクニカルメタデータをSparkのテーブルに一度保存
2. オペレーショナルメタデータをSparkのテーブルに一度保存
3. 1,2とビジネスメタデータと合わせてMysqlのテーブルに保存

そのための準備を行います。

メタデータ保存用のメタデータの保存用のテーブルとMysqlのテーブルをここで作成します。

Sparkテーブル(metadata_tmpデータベース)
- database_name -> データベース名
- table_name -> テーブル名
- table_definition ->テーブル定義
- sammary ->　テーブル説明
- row_num ->　レコード件数
- selectivity ->　セレクティビティ
- consistency_flag ->　コンシステンシー
- frequency_access ->　アクセス数

Mysqlテーブル(metadataデータベース)
- database_name -> Sparkデータベース名
- table_name -> Sparkテーブル名
- table_definition -> Sparkテーブル定義
- sammary ->　Sparkテーブル説明
- row_num ->　レコード件数
- selectivity ->　セレクティビティ
- consistency_flag ->　コンシステンシー
- frequency_access ->　アクセス数

# メタデータ保存用のMysqlテーブル

```
CREATE DATABASE if not exists metadata;
USE metadata;
CREATE TABLE if not exists metadatas(
    database_name VARCHAR(255) , 
    table_name VARCHAR(255) , 
    table_definition VARCHAR(255) , 
    sammary VARCHAR(255) , 
    row_num VARCHAR(255) , 
    selectivity VARCHAR(255) , 
    consistency_flag VARCHAR(255) , 
    frequency_access VARCHAR(255) ,
    PRIMARY KEY (database_name,table_name));
```

コーンソールに移動してMysqlコマンドを実行します。

# メタデータ保存用のSparkテーブル
ここからはメタデータ保存用のSparkテーブルを作成していきます

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()


In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

In [ ]:
# メタデータ保存用のSparkテーブル
spark.sql("drop database metadata_tmp cascade")
spark.sql("create database if not exists metadata_tmp")
spark.sql(""" 
CREATE TABLE IF NOT EXISTS metadata_tmp.sample_metadata (database_name String, table_name String,table_definition String,sammary String,record_num String,selectivity String,consistency_flag boolean,frequency_access String)
STORED AS PARQUET
LOCATION '/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/metadata_tmp.db/sample_metadata';
""")

In [ ]:
#　テーブルができたか確認します
spark.sql("show tables in metadata_tmp").show() 

# ビジネスメタデータをSparkテーブル(sample_metadata)に一度保存してみましょう
ここからはビジネスメタデータ(data_management_cruch_course.jinko_table)を一度Sparkテーブルに保存していきます。

ここで確認ですがビジネスメタデータで基本となるのは
1. テーブル定義
2. ビジネス知識（ドメイン知識）

の２つです。  
今回はこれらを格納するテーブルだと、「table_definition」と「Summary」です。  
最終的にはMysqlに入れますが、講座の都合上sample_metadataテーブルに都度格納をしながら行います。  

In [ ]:

from pyspark.sql.types import StructType, StructField, StringType, BooleanType
from pyspark.sql.functions import when

# 横着をして Spark経由でテーブル定義を取得します
table_def=spark.sql(""" show create table data_management_crush_course.jinko_table """).collect()[0].asDict()['createtab_stmt']

# カラム名、型、Null OKか否かで設定していきます
struct = StructType([
    StructField("database_name", StringType(), True),
    StructField("table_name", StringType(), True),
    StructField("table_definition", StringType(), True),
    StructField("sammary", StringType(), True),
    StructField("record_num", StringType(), True),
    StructField("selectivity", StringType(), True),
    StructField("consistency_flag", BooleanType(), True),
    StructField("frequency_access", StringType(), True)
])

df = spark.createDataFrame([(None,None,None,None,None,None,None,None)], struct)
#今回は一個づつ値を設定していくので少しトリッキーにデータフレームの値を更新していきマス。

# メタデータ取得対象のデータを更新する
df2 = df.withColumn("database_name", when(df.database_name.isNull() ,"data_management_crush_course").otherwise(df.database_name))
df2 = df2.withColumn("table_name", when(df.table_name.isNull() ,"jinko_table").otherwise(df.table_name))
df2 = df2.withColumn("table_definition", when(df.table_definition.isNull() ,table_def).otherwise(df.table_definition))
df2 = df2.withColumn("sammary", when(df.sammary.isNull() ,"一旦テーブルの説明は空にしておきます。").otherwise(df.sammary))

df2.show(truncate=False)


In [ ]:
#取得したデータをmetadata_tmp.sample_metadataに格納していきます
df2.createOrReplaceTempView("sample")

spark.sql("""
Insert overwrite  table metadata_tmp.sample_metadata 
select  * from sample
""")


In [18]:
# 結果の確認をしてみます

spark.sql("select * from metadata_tmp.sample_metadata ").show()

+--------------------+-----------+--------------------+--------------------------------------+----------+-----------+----------------+----------------+
|       database_name| table_name|    table_definition|                               sammary|record_num|selectivity|consistency_flag|frequency_access|
+--------------------+-----------+--------------------+--------------------------------------+----------+-----------+----------------+----------------+
|data_management_c...|jinko_table|CREATE TABLE `dat...|一旦テーブルの説明は空にしておきます。|       300|       6.00|           false|            null|
+--------------------+-----------+--------------------+--------------------------------------+----------+-----------+----------------+----------------+



Mon Nov 22 22:12:25 JST 2021 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Mon Nov 22 22:12:25 JST 2021 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s

In [ ]:
# Stopは忘れずに
# 忘れてしまうと、いつの間にか接続が溜まっていってしまいます
spark.stop()
spark.sparkContext.stop()